In [ ]:
import requests
import time
import json
from pprint import pprint
"""
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs, MarketOrderArgs, OrderType, OpenOrderParams, BalanceAllowanceParams, AssetType
from py_clob_client.order_builder.constants import BUY, SELL"""

GAMMA_API = "https://gamma-api.polymarket.com"
#DATA_API = "https://data-api.polymarket.com"
#CLOB_API = "https://clob.polymarket.com"

### 1. Only leagues part of filtered "leages.txt" file are considered

In [ ]:
FILE_PATH = "Prove Varie/leages.txt"
import json
def load_sport_series_mapping(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)

    mapping = {}

    for item in data:
        sport = item.get("sport")
        series = item.get("series")
        id = item.get("id")

        if not sport or not series:
            continue

        mapping[sport] = {
            "sport": sport,
            "series": int(series),
            "id": int(id)
        }

    return mapping

sport_series = load_sport_series_mapping(FILE_PATH)

print(f"Sport caricati: {len(sport_series)}")
print("Esempio:")
for k in list(sport_series.keys())[:5]:
    print(k, "->", sport_series[k])

Sport caricati: 46
Esempio:
ncaab -> {'sport': 'ncaab', 'series': 39, 'id': 1}
epl -> {'sport': 'epl', 'series': 10188, 'id': 2}
lal -> {'sport': 'lal', 'series': 10193, 'id': 3}
bun -> {'sport': 'bun', 'series': 10194, 'id': 7}
cfb -> {'sport': 'cfb', 'series': 10210, 'id': 9}


In [ ]:
from collections import defaultdict
#serieses = [10188, 10193]  # epl, lal
serieses = [v["series"] for v in sport_series.values()] # type: ignore
limit = 50
max_page = 200
events_by_series = defaultdict(list)

# -------- FETCH EVENTS (con markets già inclusi) --------
for series in serieses:
    page_id = 0
    while page_id < max_page:
        response = requests.get( # type: ignore
            f"{GAMMA_API}/events",# type: ignore
            params={
                "series_id": series,
                "active": True,
                "closed": False,
                "tag_id": 100639,
                "order": "startTime",
                "ascending": True,
                "limit": limit,
                "offset": page_id * limit
            }
        )

        events = response.json()
        if not events:
            break

        events_by_series[series].extend(events)
        page_id += 1


# -------- PRINT EVENTS + MARKETS + OUTCOMES --------
for series, events in events_by_series.items():
    print(f"\n=== SERIES {series} ({len(events)} events) ===")

    for event in events:
        event_id = event.get("id")
        event_title = event.get("title", "N/A")
        start_time = event.get("startTime", "N/A")

        print(f"\n▶ Event [{event_id}] {event_title} ({start_time})")

        markets = event.get("markets", [])

        if not markets:
            print("  (No available markets)")
            continue

        print(f"  Available markets ({len(markets)}):")

        for market in markets:
            market_id = market.get("id", "N/A")
            market_title = market.get("question", "N/A")
            outcome_prices = market.get("outcomePrices", {})

            print(
                f"    - Market [{market_id}] {market_title} | "
                f"OutcomePrices: {outcome_prices}"
            )




=== SERIES 10188 (36 events) ===

▶ Event [156530] West Ham United FC vs. Sunderland AFC (2026-01-24T12:30:00Z)
  Available markets (3):
    - Market [1157250] Will West Ham United FC win on 2026-01-24? | OutcomePrices: ["0.375", "0.625"]
    - Market [1157251] Will West Ham United FC vs. Sunderland AFC end in a draw? | OutcomePrices: ["0.295", "0.705"]
    - Market [1157252] Will Sunderland AFC win on 2026-01-24? | OutcomePrices: ["0.335", "0.665"]

▶ Event [156668] West Ham United FC vs. Sunderland AFC - More Markets (2026-01-24T12:30:00Z)
  Available markets (6):
    - Market [1157729] Spread: West Ham United FC (-1.5) | OutcomePrices: ["0.175", "0.825"]
    - Market [1157731] West Ham United FC vs. Sunderland AFC: O/U 1.5 | OutcomePrices: ["0.705", "0.295"]
    - Market [1157733] West Ham United FC vs. Sunderland AFC: O/U 2.5 | OutcomePrices: ["0.445", "0.555"]
    - Market [1157734] West Ham United FC vs. Sunderland AFC: O/U 3.5 | OutcomePrices: ["0.24", "0.76"]
    - Market [115

### 2. All the available sports in Polymarket are considered and extracted

In [18]:
# Fetch sports
response_3 = requests.get(f"{GAMMA_API}/sports")  # type: ignore
sports_data_all = response_3.json()

print(f"Found {len(sports_data_all)} sports\n")

sport_series_all = {}

for item in sports_data_all:
    sport = item.get("sport")
    series = item.get("series")
    sport_id = item.get("id")

    if not sport:
        continue

    try:
        series = int(series)
        sport_id = int(sport_id)
    except (TypeError, ValueError):
        # Skip if series or sport_id is not a number
        continue

    sport_series_all[sport] = {
        "sport": sport,
        "series": series,
        "id": sport_id
    }


Found 118 sports



In [ ]:
from collections import defaultdict

serieses_all = [v["series"] for v in sport_series_all.values()] # type: ignore
limit = 50
max_page = 200
events_by_series = defaultdict(list)

# -------- FETCH EVENTS (con markets già inclusi) --------
for series in serieses_all:
    page_id = 0
    while page_id < max_page:
        response = requests.get( # type: ignore
            f"{GAMMA_API}/events", # type: ignore
            params={
                "series_id": series,
                "active": True,
                "closed": False,
                "tag_id": 100639, # only "games" considered & not futures
                "order": "startTime",
                "ascending": True,
                "limit": limit,
                "offset": page_id * limit
            }
        )

        events = response.json()
        if not events:
            break

        events_by_series[series].extend(events)
        page_id += 1


# -------- PRINT EVENTS + MARKETS + OUTCOMES --------
for series, events in events_by_series.items():
    print(f"\n=== SERIES {series} ({len(events)} events) ===")

    for event in events:
        event_id = event.get("id")
        event_title = event.get("title", "N/A")
        start_time = event.get("startTime", "N/A")

        print(f"\n▶ Event [{event_id}] {event_title} ({start_time})")

        markets = event.get("markets", [])

        if not markets:
            print("  (No available markets)")
            continue

        print(f"  Available markets ({len(markets)}):")

        for market in markets:
            market_id = market.get("id", "N/A")
            market_title = market.get("question", "N/A")
            outcome_prices = market.get("outcomePrices", {})

            print(
                f"    - Market [{market_id}] {market_title} | "
                f"OutcomePrices: {outcome_prices}"
            )




=== SERIES 10188 (36 events) ===

▶ Event [156530] West Ham United FC vs. Sunderland AFC (2026-01-24T12:30:00Z)
  Available markets (3):
    - Market [1157250] Will West Ham United FC win on 2026-01-24? | OutcomePrices: ["0.375", "0.625"]
    - Market [1157251] Will West Ham United FC vs. Sunderland AFC end in a draw? | OutcomePrices: ["0.295", "0.705"]
    - Market [1157252] Will Sunderland AFC win on 2026-01-24? | OutcomePrices: ["0.335", "0.665"]

▶ Event [156668] West Ham United FC vs. Sunderland AFC - More Markets (2026-01-24T12:30:00Z)
  Available markets (6):
    - Market [1157729] Spread: West Ham United FC (-1.5) | OutcomePrices: ["0.175", "0.825"]
    - Market [1157731] West Ham United FC vs. Sunderland AFC: O/U 1.5 | OutcomePrices: ["0.7", "0.3"]
    - Market [1157733] West Ham United FC vs. Sunderland AFC: O/U 2.5 | OutcomePrices: ["0.445", "0.555"]
    - Market [1157734] West Ham United FC vs. Sunderland AFC: O/U 3.5 | OutcomePrices: ["0.23", "0.77"]
    - Market [1157735

For info on Event->Markets and available parameters to extract from each market: visit link "https://gamma-api.polymarket.com/events?series_id=10188&active=true&closed=false" on Chrome \
While for getting /sport data: visit "https://gamma-api.polymarket.com/sports" on Chrome \
Insider Addresses: 0xc2dba869680b8c7a580ed7d1a9bfd9b329ca86cf; 0x2577c59314e3efd45f9dd31067ea329d5a4ea5ec; 0xdbddb8bc3d3dca058a8b19c539effab1363c7417; 

The code cell below is just used to safe results into a CSV file

In [5]:
import json

def parse_outcome_prices(raw):
    """
    Convert raw outcome_prices to list of floats [YES, NO].
    Returns None if invalid.
    """
    try:
        # If it's already a list, just use it
        if isinstance(raw, list):
            return [float(raw[0]), float(raw[1])]
        # If it's a string (JSON), parse it
        elif isinstance(raw, str):
            parsed = json.loads(raw)
            return [float(parsed[0]), float(parsed[1])]
        else:
            return None
    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
        return None

import requests
import pandas as pd
from collections import defaultdict

GAMMA_API = "https://gamma-api.polymarket.com"

# -------- Initialize data list (same pattern as Kalshi) --------
data = []

# -------- FETCH SPORTS --------
response = requests.get(f"{GAMMA_API}/sports")
sports_data_all = response.json()

sport_series_all = {}

for item in sports_data_all:
    sport = item.get("sport")
    series = item.get("series")
    sport_id = item.get("id")

    if not sport:
        continue

    try:
        series = int(series)
        sport_id = int(sport_id)
    except (TypeError, ValueError):
        continue

    sport_series_all[sport] = {
        "sport": sport,
        "series": series,
        "id": sport_id
    }

serieses_all = [v["series"] for v in sport_series_all.values()]

# -------- FETCH EVENTS --------
limit = 50
max_page = 200
events_by_series = defaultdict(list)

for series in serieses_all:
    page_id = 0
    while page_id < max_page:
        response = requests.get(
            f"{GAMMA_API}/events",
            params={
                "series_id": series,
                "active": True,
                "closed": False,
                #"tag_id": 100639,  # games only
                "order": "startTime",
                "ascending": True,
                "limit": limit,
                "offset": page_id * limit
            }
        )

        events = response.json()
        if not events:
            break

        events_by_series[series].extend(events)
        page_id += 1

# -------- COLLECT EVENTS + MARKETS (FLAT, LIKE KALSHI) --------
for series, events in events_by_series.items():
    for event in events:
        event_id = event.get("id", "N/A")
        event_title = event.get("title", "N/A")
        start_time = event.get("endDate", "N/A")

        markets = event.get("markets", [])
        if not markets:
            continue

        for market in markets:
            market_id = market.get("id", "N/A")
            market_title = market.get("question", "N/A")
            outcome_prices = market.get("outcomePrices", {})

            parsed_prices = parse_outcome_prices(outcome_prices)
            if parsed_prices:
                price_yes_buy = parsed_prices[0]
                price_no_buy = parsed_prices[1]
            else:
                price_yes_buy = None
                price_no_buy = None

            data.append({
                "sport": next(
                    (k for k, v in sport_series_all.items() if v["series"] == series),
                    "N/A"
                ),
                "series_id": series,
                "event_id": event_id,
                "event_title": event_title,
                "start_time": start_time,
                "market_id": market_id,
                "market_title": market_title,
                "PriceYesBuy": price_yes_buy,
                "PriceNoBuy": price_no_buy
            })

df = pd.DataFrame(data)
rows = []
event_index = 0

# Group by sport, event_id, event_title, start_time (adjust grouping as needed)
for (sport, event_id, event_title, start_time), group in df.groupby(
    ["sport", "event_id", "event_title", "start_time"],
    dropna=False
):
    event_index += 1

    # -------- EVENT ROW --------
    rows.append({
        "row_type": "EVENT",
        "event_index": event_index,
        "sport": sport,
        "event_id": event_id,
        "event_title": event_title,
        "start_time": start_time,
        "market_id": "",
        "market_title": "",
        "PriceYesBuy": "",
        "PriceNoBuy": ""
    })

    # -------- MARKET ROWS --------
    for _, r in group.iterrows():
        rows.append({
            "row_type": "MARKET",
            "event_index": event_index,
            "sport": "",
            "event_id": "",
            "event_title": "",
            "start_time": "",
            "market_id": r["market_id"],
            "market_title": r["market_title"],
            "PriceYesBuy": r["PriceYesBuy"],
            "PriceNoBuy": r["PriceNoBuy"]
        })

# -------- FINAL CSV --------
df_hierarchical = pd.DataFrame(rows)
df_hierarchical.to_csv("polymarket_data.csv", index=False)

print(
    f"Saved polymarket_data.csv with "
    f"{event_index} events and {len(df_hierarchical)} rows."
)


Saved polymarket_data_hierarchical.csv with 2582 events and 10791 rows.
